In [ ]:
import cv2
import numpy as np
import re
from pathlib import Path
from tqdm.notebook import tqdm 


In [ ]:
rgb_path   = Path('./assets/PyMouseLifter_demo/output/raw_video.mp4') 
depth_path = Path('./assets/PyMouseLifter_demo/output/depth_vis.mp4') 
label_txt  = Path('./assets/PyMouseLifter_demo/output/behavior_classification.txt') 

out_path   = Path('./assets/PyMouseLifter_demo/output/PyMouseLifter_demo.mp4')

target_w = None   # e.g. 640
target_h = None   # e.g. 480


In [ ]:
import re, cv2, colorsys, numpy as np
from tqdm import tqdm
# ───────────────── read labels ─────────────────
pattern = re.compile(r"Frame\s+(\d+):\s+\['(.+)'\]")
labels = {}
with open(label_txt, 'r', encoding='utf-8') as f:
    for line in f:
        m = pattern.search(line)
        if m:
            labels[int(m.group(1))] = m.group(2)

# 🔹 1) behavior to color mapping  --------------------------------------------
behaviors = sorted(set(labels.values()))
n_behav   = len(behaviors)
behav2color = {}
for i, beh in enumerate(behaviors):
    # Take the HSV hue evenly, then convert it to BGR (cv2 defaults to BGR).
    h = i / max(n_behav, 1)                 # hue ∈ [0,1)
    r, g, b = colorsys.hsv_to_rgb(h, 1, 1)  # full‑sat, full‑val
    behav2color[beh] = (int(b*255), int(g*255), int(r*255))  # BGR
# ------------------------------------------------------------------------

# ───────────────── 打开视频 ─────────────────
cap_rgb   = cv2.VideoCapture(str(rgb_path))
cap_depth = cv2.VideoCapture(str(depth_path))
if not (cap_rgb.isOpened() and cap_depth.isOpened()):
    raise IOError('❌ cannot open input behavior')

n_frames = int(cap_rgb.get(cv2.CAP_PROP_FRAME_COUNT))
assert n_frames == int(cap_depth.get(cv2.CAP_PROP_FRAME_COUNT)), \
       'The frame rates of the two video streams are inconsistent!'

fps  = cap_rgb.get(cv2.CAP_PROP_FPS)
w_in = int(cap_rgb.get(cv2.CAP_PROP_FRAME_WIDTH))
h_in = int(cap_rgb.get(cv2.CAP_PROP_FRAME_HEIGHT))

# target size
target_w = target_w or w_in
target_h = target_h or h_in

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter(str(out_path), fourcc, fps,
                         (target_w * 2, target_h))

# 字体设置
font, fscale, fthick = cv2.FONT_HERSHEY_SIMPLEX, 2, 4
offset = (30, 60)

# ───────────────── loop ─────────────────
for idx in tqdm(range(n_frames), desc='Splicing'):
    ret1, frame_rgb   = cap_rgb.read()
    ret2, frame_depth = cap_depth.read()
    if not (ret1 and ret2):
        print(f'⚠️ frame {idx} read failed, terminated early.'); break

    # resize
    if (frame_rgb.shape[1], frame_rgb.shape[0]) != (target_w, target_h):
        frame_rgb = cv2.resize(frame_rgb, (target_w, target_h),
                               interpolation=cv2.INTER_CUBIC)
    if (frame_depth.shape[1], frame_depth.shape[0]) != (target_w, target_h):
        frame_depth = cv2.resize(frame_depth, (target_w, target_h),
                                 interpolation=cv2.INTER_NEAREST)

    # 🔹 2) Overlay labels (written on the depth view) --------------------------------------
    label_txt = labels.get(idx, '')
    if label_txt != 'Stationary':
        color     = behav2color.get(label_txt, (255, 255, 255))
        text_sz, _ = cv2.getTextSize(label_txt, font, fscale, fthick)
        tx = target_w - text_sz[0] - offset[0] 
        ty = offset[1]
        cv2.putText(frame_depth, label_txt, (tx, ty),
                    font, fscale, color, fthick, cv2.LINE_AA)
    # --------------------------------------------------------------------

    writer.write(np.concatenate((frame_rgb, frame_depth), axis=1))

# ───────────────── release ─────────────────
cap_rgb.release(); cap_depth.release(); writer.release()
print(f'✅ Finished, output saved at {out_path}')


拼接中: 100%|██████████| 1800/1800 [00:14<00:00, 128.56it/s]

✅ 完成，输出已保存到 assets/PyMouseLifter_demo/output/PyMouseLifter_demo.mp4
